# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
print(geoapify_key)

0cfdb17286dc4e5aa91875aeb6f17f2c


In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame using forward slashes
city_data_df = pd.read_csv("C:/Users/Nick/Downloads/Starter_Code (6)/Starter_Code/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,67.73,62,100,9.22,PN,1720578604
1,1,kodiak,57.7900,-152.4072,51.31,100,100,4.61,US,1720578605
2,2,iqaluit,63.7506,-68.5145,42.53,86,75,3.44,CA,1720578588
3,3,port-aux-francais,-49.3500,70.2167,40.51,91,99,28.21,TF,1720578608
4,4,happy valley-goose bay,53.3168,-60.3315,64.71,94,75,2.30,CA,1720578609


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:

import folium

# Configure the map plot
map = folium.Map(location=[city_data_df['Lat'].mean(), city_data_df['Lng'].mean()], zoom_start=2)
for index, row in city_data_df.iterrows():
    folium.CircleMarker(location=[row['Lat'], row['Lng']], radius=row['Humidity']/20, 
                        color='blue', fill=True, fill_color='blue', fill_opacity=0.6).add_to(map)

# Display the map
map

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:

# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = (city_data_df['Max Temp'] < 80) & (city_data_df['Max Temp'] > 70) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)

# Filter the DataFrame based on ideal weather conditions
ideal_cities_df = city_data_df[ideal_conditions]
ideal_cities_df.dropna(inplace=True)
ideal_cities_df = ideal_cities_df.dropna().copy()
# Display the DataFrame with ideal weather conditions
print(ideal_cities_df)

     City_ID                  City      Lat       Lng  Max Temp  Humidity  \
72        72            ulan bator  47.9077  106.8832     77.13        44   
95        95                 ronda  36.7423   -5.1671     70.16        31   
149      149           porto velho  -8.7619  -63.9039     77.05        78   
164      164            es castell  39.8795    4.2854     72.28       100   
260      260                mizdah  31.4451   12.9801     78.39        30   
403      403  linxia chengguanzhen  35.6003  103.2064     73.15        43   

     Cloudiness  Wind Speed Country        Date  
72            0        4.47      MN  1720578708  
95            0        3.83      ES  1720578743  
149           0        2.30      BR  1720578643  
164           0        2.30      ES  1720578843  
260           0        1.39      LY  1720578984  
403           0        1.14      CN  1720579149  


C:\Users\Nick\AppData\Local\Temp\ipykernel_25300\1366441612.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities_df.dropna(inplace=True)


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
print(hotel_df)

                     City Country      Lat       Lng  Humidity Hotel Name
72             ulan bator      MN  47.9077  106.8832        44           
95                  ronda      ES  36.7423   -5.1671        31           
149           porto velho      BR  -8.7619  -63.9039        78           
164            es castell      ES  39.8795    4.2854       100           
260                mizdah      LY  31.4451   12.9801        30           
403  linxia chengguanzhen      CN  35.6003  103.2064        43           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "hotel",
    "radius": radius,
    "limit": 1  # Limit the number of results to 1 hotel per city
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get("https://api.geoapify.com/v1/places", params=params, headers={"x-api-key": geoapify_key})

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
    # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ulan bator - nearest hotel: No hotel found
ronda - nearest hotel: No hotel found
porto velho - nearest hotel: No hotel found
es castell - nearest hotel: No hotel found
mizdah - nearest hotel: No hotel found
linxia chengguanzhen - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
72,ulan bator,MN,47.9077,106.8832,44,No hotel found
95,ronda,ES,36.7423,-5.1671,31,No hotel found
149,porto velho,BR,-8.7619,-63.9039,78,No hotel found
164,es castell,ES,39.8795,4.2854,100,No hotel found
260,mizdah,LY,31.4451,12.9801,30,No hotel found
403,linxia chengguanzhen,CN,35.6003,103.2064,43,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:

# Configure the map plot
map = folium.Map(location=[hotel_df['Lat'].mean(), hotel_df['Lng'].mean()], zoom_start=2)
for index, row in hotel_df.iterrows():
    popup_text = f"City: {row['City']}<br>Country: {row['Country']}<br>Humidity: {row['Humidity']}<br>Hotel: {row['Hotel Name']}"
    folium.CircleMarker(location=[row['Lat'], row['Lng']], radius=row['Humidity']/2, 
                        color='blue', fill=True, fill_color='blue', fill_opacity=0.6,
                        popup=popup_text).add_to(map)
# Display the map
map